# Lassa data set evaluation

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
def histograms(stats):
    # Creating histograms with custom labels
    fig, axes = plt.subplots(1, 4, figsize=(10, 3))  # Adjust grid if you have a different layout

    stats[['N_share', 'Length', 'RelativeLength', 'RelativeError']].hist(
        bins=50, log=True, ax=axes
    )

    # Setting y-axis label for the left column
    axes[0].set_ylabel('Number of sequences')

    axes[0].set_xlim(0, 1)
    axes[1].set_xlim(left=0)
    axes[2].set_xlim(left=0)
    axes[3].set_xlim(0, 1)

    # Setting x-axis labels with the feature names
    features = ['N_share', 'Length', 'RelativeLength', 'RelativeError']
    for ax, feature in zip(axes.flatten(), features):
        ax.set_xlabel(feature)
        ax.set_title('')  # Remove the title above each plot

    plt.tight_layout()
    plt.show()

# Parameters

In [ ]:
fname_stats = ''
fname_sequences = ''
outdir = ''

minlen = 2000
max_n_share = 0.01
min_relative_length = 0.95
min_ref_cov = 0.95

## Data ingestion.

We compute the following features
- the relative length of the sequence compared to the reference
- edit distance in relation to the match length
- the amount of the reference covered by the alignment
- the amount of the query covered by the alignment

In [ ]:
seqstats = pd.read_csv(fname_stats, sep='\t')
seqstats['RelativeLength'] = seqstats['Length'] / seqstats['ReferenceLength']
seqstats['RelativeError'] = seqstats['EditDistance'] / (seqstats['ReferenceEnd'] - seqstats['ReferenceStart'])
seqstats['ReferenceCoverage'] = (seqstats['ReferenceEnd'] - seqstats['ReferenceStart']) / seqstats['ReferenceLength']
seqstats['QueryCoverage'] = (seqstats['QueryEnd'] - seqstats['QueryStart']) / seqstats['Length']

seqstats

Overall length and N_share distribution before any filtering has happened.

In [ ]:
seqstats[['Length', 'N_share']].hist(bins=50, log=True)

## Removing unaligned sequences.

We remove all sequences that were not mapped. Let's have a look at the 10 largest sequences first.

In [ ]:
unmapped = seqstats[seqstats['IsForward'].isna()][['Sequence', 'Length', 'N_share']]
print(f'Number of unmapped reads={len(unmapped)}')
unmapped.nlargest(10, ['Length'])

In [ ]:
unmapped.hist(bins=20, log=True)

In [ ]:
seqstats_mapped = seqstats.dropna(subset='Reference')

## Removing supplementary alignments

Additional to the best hit, minimap2 also sometimes computes supplementary alignments. We remove them here.

In [ ]:
seqstats_primary = seqstats_mapped[seqstats_mapped['IsSupplementaryAlignment'] == False]

## Remove short sequences and those with too many Ns

In [ ]:
seqstats_minlen = seqstats_mapped[seqstats_mapped['Length'] >= minlen]
seqstats_high_coverage = seqstats_minlen[seqstats_minlen['N_share'] <= max_n_share]

print('Before filtering:', len(seqstats_mapped))
print(f'N-share larger than {max_n_share}:', len(seqstats_mapped[seqstats_mapped['N_share'] > max_n_share]))
print(f'Length smaller than {minlen}:', len(seqstats_mapped[seqstats_mapped['Length'] < minlen]))
print('After filtering:', len(seqstats_high_coverage))

## Remove too short genomes
We remove the genomes that are too short compared to the reference genome.

In [ ]:
seqstats_long_seqs = seqstats_high_coverage[seqstats_high_coverage['RelativeLength'] > min_relative_length]
print(
    f'{len(seqstats_long_seqs)} of {len(seqstats_high_coverage)} sequences '
    f'remain after filtering for {min_relative_length} minimum relative length'
)

Additionally we remove all sequences, where the alignment does not cover enough of the target genome.

In [ ]:
seqstats_long_seqs.nsmallest(10, ['ReferenceCoverage'])

In [ ]:
seqstats_min_ref_cov = seqstats_long_seqs[seqstats_long_seqs['ReferenceCoverage'] >= min_ref_cov]
print(f'Keeping {len(seqstats_min_ref_cov)} of {len(seqstats_long_seqs)} sequences')

## Per reference genome counts
Let's have a look what is left after filtering with respect to the distinct reference genomes.

In [ ]:
ref_counts_raw = seqstats_primary['Reference'].value_counts()
ref_counts_minlen = seqstats_minlen['Reference'].value_counts()
ref_counts_high_cov = seqstats_high_coverage['Reference'].value_counts()
ref_counts_filtered = seqstats_min_ref_cov['Reference'].value_counts()
pd.DataFrame({'Raw': ref_counts_raw, 'MinLen': ref_counts_minlen, 'MaxN': ref_counts_high_cov,  'Filtered': ref_counts_filtered})

## Statistics after filtering

In [ ]:
histograms(seqstats_min_ref_cov)

## Read orientation
Finally, lets get the filtered reads with harmonized orientation sorted into the Segments.

In [ ]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import os

df_filtered_final = seqstats_min_ref_cov
os.makedirs(outdir, exist_ok=True)

segments = sorted(set(df_filtered_final['Segment'])) 
fname_template_out = os.path.join(outdir, "{}.fasta")

outfiles = {
    segment: open(fname_template_out.format(segment), 'w')
    for segment in segments
}

segment_and_orientation = {
    row['Sequence']: (row['IsForward'], row['Segment'])
    for _, row in df_filtered_final.iterrows()
}

for record in SeqIO.parse(fname_sequences, "fasta"):
    if record.id not in segment_and_orientation:
        continue
    is_forward, segment = segment_and_orientation[record.id]
    if is_forward is None:
        raise RuntimeError(f"Invalid orientation {is_forward}")
    orientation = 'forward' if is_forward else 'reverse'
    new_record = SeqRecord(
        seq=record.seq if is_forward else record.seq.reverse_complement(),
        id=record.id,
        name=record.name,
        description=f'{record.description}|{orientation}|{segment}'
    )
    SeqIO.write(new_record, outfiles[segment], "fasta")

for outfile in outfiles.values():
    outfile.close()

In [ ]:
# hits_merge = seqstats_mapped.groupby('Sequence').agg(
#     N_alignments=('IsForward', 'count'),
#     SameOrientation=('IsForward', lambda x: x.nunique() == 1),
#     SameReference=('Reference', lambda x: x.nunique() == 1),
#     SequenceLength=('Length', 'max'),
# ).reset_index()
# #hits_merge[(hits_merge['SameOrientation'] == False) | (hits_merge['SameReference'] == False)]
# hits_merge[hits_merge['N_alignments'] > 1]